In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
import textgrad as tg
# import openai
# import textgrad as tg
# import os

## Textgrad

In [2]:
# ... existing code ...


# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Set the backward engine for TextGrad
tg.set_backward_engine("gpt-3.5-turbo")

# Define the LLM
llm = tg.BlackboxLLM("gpt-3.5-turbo")

# Define the prompt as a tg.Variable
prompt = tg.Variable("Write a short story about a robot who learns to feel emotions.", 
role_description="story prompt",
requires_grad=True)

# Define the optimizer without prompt_template
optimizer = tg.TGD(parameters=[prompt])

# Define a loss function (example - you'll need to define your actual loss)
# For instance, if you want to optimize the prompt to generate a story that is 'heartwarming':
loss_instruction = "Evaluate the story for how heartwarming it is. Provide a score from 1 to 10, where 10 is extremely heartwarming."
loss_fn = tg.TextLoss(loss_instruction)

# Generate an initial story using the prompt
story = llm(prompt)

# Calculate loss and optimize (example loop)
for i in range(5):
    loss = loss_fn(story)
    loss.backward()
    optimizer.step()
    print(f"Iteration {i+1}, Loss: {loss.value}")
    story = llm(prompt) # Regenerate story with optimized prompt

print("\nOptimized Prompt:", prompt.value)
print("\nOptimized Story:", story.value)

# ... existing code ...

Iteration 1, Loss: I would rate this story a 8 out of 10 for heartwarming. The transformation of R-37 from a cold, emotionless robot to a being capable of love, empathy, and understanding is touching and heartening. The message of embracing emotions and finding strength in vulnerability is powerful and uplifting. The story's focus on connections, friendships, and personal growth adds to its heartwarming nature.
Iteration 2, Loss: I would rate this story a 9 out of 10 for heartwarming. The transformation of Aria from a purely efficient robot to a being capable of empathy, compassion, and love is truly touching. The journey of self-discovery, the realization of the importance of emotions, and the ultimate acceptance of her newfound emotional depth all contribute to a heartwarming and uplifting narrative. The themes of growth, understanding, and connection resonate deeply, making this story a heartwarming tale of transformation and acceptance.
Iteration 3, Loss: I would rate this story a 

## dspy

https://dspy.ai/#__tabbed_3_1

In [2]:
import dspy
import os

# Set up your OpenAI API key
# Make sure to replace 'YOUR_OPENAI_API_KEY' with your actual key
# It's recommended to load this from an environment variable
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# Configure the language model (e.g., OpenAI's GPT-3.5 Turbo)
# You can choose other models like dspy.OpenAI(model='gpt-4') or dspy.Ollama for local models
llm = dspy.LM(model='openai/gpt-3.5-turbo', max_tokens=500) # Corrected line
dspy.settings.configure(lm=llm)

# 1. Define the Signature for your task
# A Signature describes the input and output fields of a DSPy module.
class StoryGeneration(dspy.Signature):
    """Generates a short story based on a given genre and theme."""
    genre: str = dspy.InputField(desc="the genre of the story (e.g., fantasy, sci-fi, mystery)")
    theme: str = dspy.InputField(desc="the central theme or idea of the story")
    story: str = dspy.OutputField(desc="a compelling short story")

# 2. Define the DSPy Module
# This module uses the StoryGeneration signature to predict a story.
class StoryGenerator(dspy.Module):
    def __init__(self):
        # Call the parent class (dspy.Module) constructor to properly initialize the object
        super().__init__()
        self.generate_story = dspy.Predict(StoryGeneration)

    def forward(self, genre, theme):
        prediction = self.generate_story(genre=genre, theme=theme)
        return prediction

# 3. Create an instance of your module
story_program = StoryGenerator()

# 4. Run the program
genre = "fantasy"
theme = "a lost ancient artifact"

print(f"\nGenerating a {genre} story about '{theme}'...")
response = story_program(genre=genre, theme=theme)

print("\nGenerated Story:")
print(response.story)

# To optimize this program, you would typically use a DSPy optimizer like BootstrapFewShot
# For example:
# from dspy.teleprompt import BootstrapFewShot
# optimizer = BootstrapFewShot(metric=your_metric_function)
# optimized_program = optimizer.compile(story_program, trainset=your_training_data)
# print(optimized_program(genre=genre, theme=theme).story)

# ... existing code ...


Generating a fantasy story about 'a lost ancient artifact'...

Generated Story:
In the mystical land of Eldoria, rumors spread like wildfire about a lost ancient artifact that possessed unimaginable power. Many brave adventurers had set out on a quest to find it, but none had returned. One day, a young mage named Elara stumbled upon an ancient map that supposedly led to the artifact's hidden location.

Determined to uncover the truth behind the artifact, Elara embarked on a perilous journey through dark forests, treacherous mountains, and winding caves. Along the way, she faced fierce magical creatures and cunning traps set by those who sought to protect the artifact's secrets.

After facing countless challenges and overcoming impossible odds, Elara finally reached the heart of the ancient ruins where the artifact was said to be hidden. As she laid her hands on the artifact, a surge of power coursed through her veins, filling her with knowledge and wisdom beyond her wildest dreams.

B

## Not worked example

In [ ]:
# Load your LLM (e.g., OpenAI)
# Replace with your actual API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Define your initial prompt
initial_prompt = "Given the following text, answer the question: {text}\nQuestion: {question}\nAnswer:"

# Define a simple evaluation function (e.g., exact match)
def exact_match_eval(inputs, ground_truth):
    if inputs["prediction"] == ground_truth:
        return 1.0
    else:
        return 0.0

# Example dataset (replace with your actual data)
data = [
    {
        "text": "The Eiffel Tower is in Paris.",
        "question": "Where is the Eiffel Tower?",
        "answer": "Paris"
    }
]

# Create a TextGrad optimizer
optimizer = tg.TGD(
    prompt_template=initial_prompt,
    eval_fn=exact_match_eval,
    model="gpt-3.5-turbo",  # Pass model name directly as a string
    dataset=data
)


# Optimize the prompt
optimized_prompt = optimizer.optimize(iterations=2)
print(optimized_prompt)

TypeError: TextualGradientDescent.__init__() got an unexpected keyword argument 'prompt_template'